In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import time

In [2]:
test_pickle = 'test.pickle'
# test_pickle = 'norm_test.pickle'
with open(test_pickle, 'rb') as inFile:
    content = pickle.load(inFile)
    X_test = content['X_test']
    y_test = content['y_test']
    print('Test set', X_test.shape, y_test.shape)

Test set (5000, 32, 32, 1) (5000, 6)


In [3]:
valid_pickle = 'valid.pickle'
with open(valid_pickle, 'rb') as inFile:
    content = pickle.load(inFile)
    X_valid = content['X_valid']
    y_valid = content['y_valid']
    print('Validation set', X_valid.shape, y_valid.shape)

Validation set (1000, 32, 32, 1) (1000, 6)


In [4]:
pickle_file = 'SVHN.pickle'
# pickle_file = 'toy_data.pickle'
with open(pickle_file, 'rb') as inFile:
    content = pickle.load(inFile)
    #X_train = content['toy_X_train']
    #y_train = content['toy_y_train']
    #X_valid = content['toy_X_valid']
    #y_valid = content['toy_y_valid']
    #X_test = content['toy_X_test']
    #y_test = content['toy_y_test']
    
    X_train = content['X_train']
    y_train = content['y_train']
    #X_valid = content['X_valid']
    #y_valid = content['y_valid']
    #X_test =  content['X_test']
    #y_test =  content['y_test']
    del content
    print('Train set', X_train.shape, y_train.shape)
    #print('Validation set', X_valid.shape, y_valid.shape)
    #print('Test set', X_test.shape, y_test.shape)

Train set (229754, 32, 32, 1) (229754, 6)


In [5]:
print(y_test.dtype, y_train.dtype, y_valid.dtype)

float32 float32 float32


In [6]:
y_test = y_test.astype('int32', copy=False)
y_train = y_train.astype('int32', copy=False)
y_valid = y_valid.astype('int32', copy=False)

print(y_test.dtype, y_train.dtype, y_valid.dtype)

int32 int32 int32


In [7]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels[:,1:]) / (5*len(labels)))

In [6]:
#def accuracy(predictions, labels):
#    print(predictions.shape)
#    print(labels[:,1].shape)
#    print(np.argmax(predictions, 1).T.shape)
#    return (100.0 * np.sum(np.argmax(predictions, 1).T == labels[:,1]) / len(labels))

In [16]:
img_size = 32
n_labels = 11
n_channels = 1
batch_size = 64
# n_digits = 1
keep_prob = 1.0
kernel_size = 3
depth1, depth2, depth3 = 16, 32, 64
n_nodes_full = 64

graph = tf.Graph()
with graph.as_default():
    
    # train, validation and test data
    with tf.name_scope("input_data") as scope:
        tf_train_dataset = tf.placeholder(
            tf.float32, shape=(batch_size, img_size, img_size, n_channels))
        tf_train_labels = tf.placeholder(tf.int32, shape = (batch_size, 6))
        tf_valid_dataset = tf.constant(X_valid)
        tf_test_dataset = tf.constant(X_test)
    
    # variables
    with tf.name_scope("weights_and_biases") as scope:
        w1_conv = tf.get_variable('conv1',
                                   shape=[kernel_size,kernel_size,n_channels,depth1],
                                   initializer=tf.contrib.layers.xavier_initializer_conv2d())
        b1_conv = tf.Variable(tf.constant(1.0, shape=[depth1], name='conv1_biases'))
    
    
        w2_conv = tf.get_variable('conv2',
                                   shape=[kernel_size,kernel_size,depth1,depth2],
                                   initializer=tf.contrib.layers.xavier_initializer_conv2d())
        b2_conv = tf.Variable(tf.constant(1.0, shape=[depth2], name='conv2_biases'))
        
        
        w3_conv = tf.get_variable('conv3',
                                   shape=[kernel_size,kernel_size,depth2,depth3],
                                   initializer=tf.contrib.layers.xavier_initializer_conv2d())
        b3_conv = tf.Variable(tf.constant(1.0, shape=[depth3], name='conv3_biases'))
   
    
        w_fc = tf.get_variable('fc_weights', shape=[8*8*64, n_nodes_full],
                             initializer=tf.contrib.layers.xavier_initializer())
        b_fc = tf.Variable(tf.constant(1.0, shape=[n_nodes_full], name='fc_biases'))
    
    
        w1_out = tf.get_variable('out_layer1', shape=[n_nodes_full, n_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
        b1_out = tf.Variable(tf.constant(1.0, shape=[n_labels], name='out_layer1_biases'))
    
    
        w2_out = tf.get_variable('out_layer2', shape=[n_nodes_full, n_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
        b2_out = tf.Variable(tf.constant(1.0, shape=[n_labels], name='out_layer2_biases'))
    
    
    
        w3_out = tf.get_variable('out_layer3', shape=[n_nodes_full, n_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
        b3_out = tf.Variable(tf.constant(1.0, shape=[n_labels], name='out_layer3_biases'))
    
    
        w4_out = tf.get_variable('out_layer4', shape=[n_nodes_full, n_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
        b4_out = tf.Variable(tf.constant(1.0, shape=[n_labels], name='out_layer4_biases'))
    
    
        w5_out = tf.get_variable('out_layer5', shape=[n_nodes_full, n_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
        b5_out = tf.Variable(tf.constant(1.0, shape=[n_labels], name='out_layer5_biases'))
        
        
    #with tf.name_scope('BN') as scope:
    #    gamma = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    #    tf.Variable(tf.truncated_normal([img_size*img_size, n_nodes]))
    
    
    def model(dataset, keep_prob):
        with tf.name_scope("Conv1") as scope:
            conv_1 = tf.nn.conv2d(dataset, w1_conv, [1,1,1,1],'SAME') + b1_conv
            # lrn1 = tf.nn.lrn(conv_1)
            # print('conv 1 shape is:', conv_1.get_shape().as_list())
            relu_1 = tf.nn.relu(conv_1)
            pool_1 = tf.nn.max_pool(relu_1, [1,2,2,1], [1,2,2,1], 'SAME')
            #print('pool_1 shape is: ', pool_1.get_shape().as_list())
        with tf.name_scope("Conv2") as scope:
            conv_2 = tf.nn.conv2d(relu_1, w2_conv, [1,1,1,1],'SAME') + b2_conv
            # lrn2 = tf.nn.lrn(conv_2)
            relu_2 = tf.nn.relu(conv_2)
            pool_2 = tf.nn.max_pool(relu_2, [1,2,2,1], [1,2,2,1], 'SAME')
            #print('pool_2 shape is: ', pool_2.get_shape().as_list())
        with tf.name_scope("Conv3") as scope:
            conv_3 = tf.nn.conv2d(pool_2, w3_conv, [1,1,1,1], 'SAME') + b3_conv
            # lrn3 = tf.nn.lrn(conv_3)
            relu_3 = tf.nn.relu(conv_3)
            pool_3 = tf.nn.max_pool(relu_3, [1,2,2,1], [1,2,2,1], 'SAME')
            #print('pool_3 shape is: ', pool_3.get_shape().as_list())
            pool_3 = tf.reshape(pool_3, (-1, 8*8*64))
            #print('after reshape, the shape is: ', pool_3.get_shape().as_list())
            #print('full connected layer shape is:', w_fc.get_shape().as_list())
        with tf.name_scope("FC") as scope:
            fc = tf.nn.relu(tf.matmul(pool_3, w_fc) + b_fc)
            fc_dropped = tf.nn.dropout(fc, keep_prob)
        with tf.name_scope("out_put") as scope:
            #print('out put layer shape is:', w1_out.get_shape().as_list())
            logits1 = tf.matmul(fc_dropped, w1_out) + b1_out
            logits2 = tf.matmul(fc_dropped, w2_out) + b2_out
            logits3 = tf.matmul(fc_dropped, w3_out) + b3_out
            logits4 = tf.matmul(fc_dropped, w4_out) + b4_out
            logits5 = tf.matmul(fc_dropped, w5_out) + b5_out
            #print('logits shape is:', logits5.get_shape().as_list())
        return logits1, logits2, logits3, logits4, logits5
    # training computation.
    # with tf.name_scope("Loss_function") as scope:
    # logits = model(tf_train_dataset, 1.0)
        # print('logit shape is', logits[4].get_shape().as_list())
        # print('train label shape is',tf_train_labels[:,5].get_shape().as_list())
    # loss_1 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[0], tf_train_labels[:,1]))
    # loss_2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[1], tf_train_labels[:,2]))
    # loss_3 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[2], tf_train_labels[:,3]))
    # loss_4 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[3], tf_train_labels[:,4]))
    # loss_5 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[4], tf_train_labels[:,5]))
    logits1, logits2, logits3, logits4, logits5 = model(tf_train_dataset, keep_prob)
    #print(tf_train_labels[:,5].get_shape().as_list())
    loss = tf.reduce_mean(
               tf.nn.sparse_softmax_cross_entropy_with_logits(logits1, tf_train_labels[:,1])) \
          +tf.reduce_mean(
               tf.nn.sparse_softmax_cross_entropy_with_logits(logits2, tf_train_labels[:,2])) \
          +tf.reduce_mean(
               tf.nn.sparse_softmax_cross_entropy_with_logits(logits3, tf_train_labels[:,3])) \
          +tf.reduce_mean(
               tf.nn.sparse_softmax_cross_entropy_with_logits(logits4, tf_train_labels[:,4])) \
          +tf.reduce_mean(
               tf.nn.sparse_softmax_cross_entropy_with_logits(logits5, tf_train_labels[:,5]))
    tf.scalar_summary('LossValue',loss)
    # loss = loss_1 + loss_2 + loss_3 + loss_4 + loss_5
    
    # Optimizer
    with tf.name_scope("Optimizer") as scope:
        global_step = tf.Variable(0)
        learning_rate = tf.train.exponential_decay(0.01, global_step, 250, 0.95)
        tf.scalar_summary('LearningRate',learning_rate)
        # optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
    # with tf.name_scope("Accuracy") as scope:
    #    accu = accuracy(train_prediction, tf_train_labels[:,1:])
    #    tf.scalar_summary('train_accuracy', accu)
    
    # Predictions of train, validation and test datasets
    with tf.name_scope("Train_prediction") as scope:
        train_logits = model(tf_train_dataset, keep_prob)
        # print(train_logits[0].get_shape().as_list())
        train_prediction = tf.pack(map(tf.nn.softmax, train_logits))
        # train_prediction = tf.nn.softmax(train_logits)
        # print(train_prediction.get_shape().as_list())
    with tf.name_scope("Validation_prediction") as scope:
        valid_logits = model(tf_valid_dataset, 1.0)
        valid_prediction = tf.pack(map(tf.nn.softmax, valid_logits))
        # valid_prediction = tf.nn.softmax(valid_logits)
    with tf.name_scope("Test_prediction") as scope:
        test_logits = model(tf_test_dataset, 1.0)
        test_prediction = tf.pack(map(tf.nn.softmax, test_logits))
        # test_prediction = tf.nn.softmax(valid_logits)
    
    # with tf.name_scope('Accuracy') as scope:
    #    print(train_prediction.get_shape().as_list())
    #    print(np.argmax(train_prediction.eval(), 2).T.shape)
    #    train_accu = accuracy(train_prediction, )
    #    tf.scalar_summary('Accuracy',train_accu)
    
    # valid_preds = tf.nn.softmax(model(tf_valid_dataset, 1.0))
    # test_preds = tf.nn.softmax(model(tf_test_dataset, 1.0))
    
    summary_op = tf.merge_all_summaries()
    
    # Sheldon is in charge of saving and restore the model
    Sheldon = tf.train.Saver(max_to_keep=3)

In [17]:
num_steps = 10001
model_name = 'convonet_model_no_LCN.mdl'

accus = []

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()  

    # Sheldon.restore(session, model_name)
    # print("Model restored.")
    writer = tf.train.SummaryWriter('/tmp/summaries/run5', session.graph)
    
    print('Initialized')
    t1 = time.time()
    for step in range(num_steps):
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        batch_data = X_train[offset:(offset + batch_size), :, :, :]
        batch_labels = y_train[offset:(offset + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, summary = session.run(
                [optimizer, loss, train_prediction, summary_op], feed_dict=feed_dict)
        
        writer.add_summary(summary, step)

        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            #if step == 0:
                #print(type(predictions), predictions.shape)
                #np.argmax(predictions, 1).T.shape
            mini_accu = accuracy(predictions, batch_labels)
            print('Minibatch accuracy: %.5f%%' % mini_accu)
            v_accu = accuracy(valid_prediction.eval(), y_valid)
            accus.append((mini_accu, v_accu))
            print('Validation accuracy: %.5f%%' % v_accu)
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), y_test))
    writer.flush()
    writer.close()
    path = Sheldon.save(session, model_name)
    print('Model saved in {}'.format(path))
    t2 = (time.time() - t1)/60
    print('Training time is {} mins'.format(t2))

Initialized
Minibatch loss at step 0: 184.367783
Minibatch accuracy: 5.31250%
Validation accuracy: 30.42000%
Minibatch loss at step 500: 5.067674
Minibatch accuracy: 66.25000%
Validation accuracy: 64.78000%
Minibatch loss at step 1000: 4.270798
Minibatch accuracy: 72.18750%
Validation accuracy: 72.48000%
Minibatch loss at step 1500: 3.552061
Minibatch accuracy: 75.00000%
Validation accuracy: 76.24000%
Minibatch loss at step 2000: 3.736381
Minibatch accuracy: 75.62500%
Validation accuracy: 78.16000%
Minibatch loss at step 2500: 3.605282
Minibatch accuracy: 78.43750%
Validation accuracy: 79.60000%
Minibatch loss at step 3000: 3.023263
Minibatch accuracy: 80.62500%
Validation accuracy: 80.32000%
Minibatch loss at step 3500: 3.506025
Minibatch accuracy: 77.50000%
Validation accuracy: 81.48000%
Minibatch loss at step 4000: 2.558393
Minibatch accuracy: 81.25000%
Validation accuracy: 81.52000%
Minibatch loss at step 4500: 3.374576
Minibatch accuracy: 77.50000%
Validation accuracy: 82.16000%
M

In [15]:
steps = np.arange(21)*500
# steps = np.arange(len(accus))*100
mini_acc = map(lambda t: t[0], accus)
valid_acc = map(lambda t: t[1], accus)
plt.figure(figsize=(16,9), dpi=75)
plt.scatter(steps, mini_acc)
plt.scatter(steps, valid_acc, c='r')

plt.title('Accuracy Over Train Steps')
plt.legend(['mini batch accuracy', 'validation accuray'])
plt.xlabel('train steps')
plt.ylabel('accuracy')
plt.show()

In [56]:
print(steps, len(accus))

[    0   500  1000  1500  2000  2500  3000  3500  4000  4500  5000  5500
  6000  6500  7000  7500  8000  8500  9000  9500 10000] 21
